# workflow.ipynb

In [14]:
import os
import glob
import shutil
import pandas as pd

In [19]:
CATALOG_PATH = '/home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv'

## qrcode_generator.ipynb

Generates small QRcode labels containing sequrntial serial numbers. 
Each label is placed at the center of a sticky trap to uniquely identify the trap.
The labelalso acts as a target which facilitates autofocus.
**get_image.py** can read the QRcode.
```
Usage: papermill [OPTIONS] NOTEBOOK_PATH [OUTPUT_PATH]

Parameters inferred for notebook '/home/aubrey/Desktop/sticky_trap_imaging/code/qrcode_generator.ipynb':
  FONT_SIZE: Unknown type (default 4)
                                  points
  FONT: Unknown type (default 'helvetica')
  YMIN: Unknown type (default 1)
                                  inches
  LABEL_WIDTH: Unknown type (default 0.5)
                                  inches
  LABEL_HEIGHT: Unknown type (default 0.5)
                                  inches
  XMIN: Unknown type (default 0.5)
                                  inches
  NROWS: Unknown type (default 10)
  NCOLS: Unknown type (default 10)
  PRINT: Unknown type (default False)
  MIN_SN: Unknown type (default 1)
  MAX_SN: Unknown type (default 100)
  PAGE_HEADER: Unknown type (default 'serial numbers generated by qrcode_generator.ipynb')
  PDF_FILENAME: Unknown type (default 'qrcodes.pdf')
```

## get_image.ipynb
Gets a single sticky trap image from Raspberry Pi 16MP autofocus camera.
```
Parameters inferred for notebook '/home/aubrey/Desktop/sicky_trap_imaging/code/get_image.ipynb':
  RPI_IP: Unknown type (default "192.168.1.102")
  RPI_USERNAME: Unknown type (default "pi")
  RPI_PASSWORD: Unknown type (default "canada12")
  TRAP_ID: Unknown type (default 999)
  USE_QRCODE: Unknown type (default True)
  OUTPUT_DIR: Unknown type (default './tinian_cycads')
```

## crop_square_centered_on_qrcode.ipynb
Crops a single sticky trap image using the QRcode as a center point.
```
ORIGINAL_IMAGE_PATH = '/home/aubrey/Desktop/tinian_sticky_trap_images/1/original.png'
CROPPED_IMAGE_PATH =  '/home/aubrey/Desktop/tinian_sticky_trap_images/1/cropped.png'
CROP_WIDTH = 2500
```

## extract_insect_images.ipynb
Detects insects and other particles on a sticky trap image. Each insect/particle is saved in OUTPUTPATH.
```
  IMAGEPATH: Unknown type (default '/home/aubrey/Desktop/my_sticky_trap.jpg')
  OUTPUTPATH: Unknown type (default 'my_sticky_trap')
  CANNY: Unknown type (default 45)
```

## clustimage
```
Parameters inferred for notebook '/home/aubrey/Desktop/sticky_trap_imaging/code/my_clustimage.ipynb':
  FILE_LIST_GLOB: Unknown type (default '/home/aubrey/Desktop/libcamera/tinian_cycads_cropped/40/0*a.png')
```

# TODO
* remove rotation in sticky_trap_analysis.ipynb
* rename sticky_trap_analysis.ipynb to extract_bug_images.ipynb
* move Jupyter notebooks into sticky_trap_imaging/code

```
<parent>
    <trap_id>
        original.png
        cropped.png
        map.png
        1.png
        2.png
        ...
        n.png
```
* **original.png** generated by **get_image.ipynb**
* **cropped.png** generated by **crop_square_centered_on_qrcode.ipynb**
* **map.png, 1.png, 2.png** generated by **extract_insect_images.ipynb**

In [6]:
DATADIR = '/home/aubrey/Desktop/tinian_sticky_trap_images'
ORIGINALDIR = '/home/aubrey/Desktop/libcamera/tinian_cycads'

if os.path.exists(DATADIR):
    print(f'Deleting {DATADIR}.')
    shutil.rmtree(DATADIR)

In [7]:
print(f'Copying images from {ORIGINALDIR} to {DATADIR}.')

# Create data directory
if not os.path.exists(DATADIR):
    os.makedirs(DATADIR)

# Get list of original files
original_files = glob.glob(f'{ORIGINALDIR}/*')
                           
# Remove two unnedded items from list                        
original_files.remove(f'{ORIGINALDIR}/40.png')
original_files.remove(f'{ORIGINALDIR}/scale.png')
                      
for original_file in original_files:

    # Create directory
    sn = os.path.basename(original_file).replace('.png', '')
    if not os.path.exists(f'{DATADIR}/{sn}'):
        os.mkdir(f'{DATADIR}/{sn}')
    
    # Copy original_file to {DATADIR}/{sn}/original.png
    command = f'cp {original_file} {DATADIR}/{sn}/original.png'
    os.system(command)

Copying images from /home/aubrey/Desktop/libcamera/tinian_cycads to /home/aubrey/Desktop/tinian_sticky_trap_images.


In [8]:
print('Cropping original images.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/crop_square_centered_on_qrcode.ipynb'
OUTPUT_NOTEBOOK = 'temp.ipynb'
CROP_WIDTH = 2500
for sn in range(1, 25):
    ORIGINAL_IMAGE_PATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/original.png'
    CROPPED_IMAGE_PATH =  f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/cropped.png'
    print(f'   Processing{ORIGINAL_IMAGE_PATH}.')
    command = f'papermill {JUPYTER_NOTEBOOK} {OUTPUT_NOTEBOOK} -p ORIGINAL_IMAGE_PATH {ORIGINAL_IMAGE_PATH} -p CROPPED_IMAGE_PATH {CROPPED_IMAGE_PATH} -p CROP_WIDTH {CROP_WIDTH}'
    os.system(command)

Cropping original images.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/1/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/2/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/3/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/4/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/5/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/6/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/7/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/8/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/9/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/10/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/11/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images/12/original.png.
   Processing/home/aubrey/Desktop/tinian_sticky_trap_images

In [9]:
print('Extracting insect images.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/extract_insect_images.ipynb'
OUTPUT_NOTEBOOK = 'temp.ipynb'
CANNY = 45
for sn in range(1, 25):
    IMAGEPATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/cropped.png'
    OUTPUTPATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}'
    print(f'   Processing {IMAGEPATH}.')
    command = f'papermill {JUPYTER_NOTEBOOK} {OUTPUT_NOTEBOOK} -p IMAGEPATH {IMAGEPATH} -p OUTPUTPATH {OUTPUTPATH} -p CANNY {CANNY}'
    os.system(command)

Extracting insect images.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/1/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/2/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/3/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/4/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/5/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/6/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/7/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/8/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/9/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/10/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/11/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_images/12/cropped.png.
   Processing /home/aubrey/Desktop/tinian_sticky_trap_image

In [41]:
# Run clustimage for each size_class. Cluster will be added to catalog.

print('Running clustimage.ipynb.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb'
OUTPUT_NOTEBOOK = 'temp.ipynb'
# FILE_LIST_GLOB = '/home/aubrey/Desktop/tinian_sticky_trap_images/*/[0-9][0-9][0-9].png'

pd.set_option('max_colwidth', 0)
df = pd.read_csv(CATALOG_PATH)
size_classes = df['size_class'].unique().tolist()

for size_class in size_classes:
    size_class = size_class.replace(' ', '-')
    command = f'papermill {JUPYTER_NOTEBOOK} {OUTPUT_NOTEBOOK} -p CATALOG_PATH {CATALOG_PATH} -p SIZE_CLASS {size_class}'
    print(command)
    os.system(command)

Running clustimage.ipynb.
papermill /home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb temp.ipynb -p CATALOG_PATH /home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv -p SIZE_CLASS 0.1-to-0.2-mm
papermill /home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb temp.ipynb -p CATALOG_PATH /home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv -p SIZE_CLASS 0.3-to-0.4-mm
papermill /home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb temp.ipynb -p CATALOG_PATH /home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv -p SIZE_CLASS 0.4-to-0.5-mm
papermill /home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb temp.ipynb -p CATALOG_PATH /home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv -p SIZE_CLASS 0.2-to-0.3-mm
papermill /home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb temp.ipynb -p CATALOG_PATH /home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv -p SIZE_CLASS 0.7-to-0.8-mm
papermill /home/aubrey/Desktop/sticky_t

In [11]:
print('Finished')

Finished
